In [2]:
import os
os.chdir("/gpfs/home/asun/jin_lab/GEARS/0_boli/")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

/gpfs/group/jin/asun/GEARS/0_boli


In [2]:
import scanpy as sc
adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/raw_data/seur.cur.h5ad')
adata

AnnData object with n_obs × n_vars = 35527 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident'
    var: 'variable_gene'

In [29]:
print(adata.to_df().iloc[:5, :5])

                                            Rp1  Sox17  Gm37323  Rgs20  Oprk1
batch1_samp1_chan1_AAACCTGAGACAAAGG-1  0.000000    0.0      0.0    0.0    0.0
batch1_samp1_chan1_AAACCTGAGGCCGAAT-1  0.730666    0.0      0.0    0.0    0.0
batch1_samp1_chan1_AAACCTGCAACTGGCC-1  0.000000    0.0      0.0    0.0    0.0
batch1_samp1_chan1_AAACCTGCACGGTAAG-1  0.000000    0.0      0.0    0.0    0.0
batch1_samp1_chan1_AAACCTGCACTATCTT-1  0.000000    0.0      0.0    0.0    0.0


In [17]:
adata_raw = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/raw_data/seur.cur.h5ad')
adata.raw = adata_raw

In [28]:
adata

AnnData object with n_obs × n_vars = 35527 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident'
    var: 'variable_gene', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [38]:
import numpy as np

gene_name = 'Tafa1'  # replace with your gene of interest
cell_type = 'L6 CT CTX'  # your cell type of interest

# Get gene index
gene_idx = adata.var_names.get_loc(gene_name)

# Filter cells by condition AND cell type
perturb_cells = adata[(adata.obs['Assign'] == 'Tbr1_0') & (adata.obs['predicted.subclass'] == cell_type)]
control_cells = adata[(adata.obs['Assign'] == 'NT_0') & (adata.obs['predicted.subclass'] == cell_type)]

# Calculate mean expression for the gene in each group
mean_perturb = perturb_cells.X[:, gene_idx].mean()
mean_control = control_cells.X[:, gene_idx].mean()

# Handle sparse matrix if needed
if hasattr(mean_perturb, "toarray"):
    mean_perturb = mean_perturb.toarray().ravel()[0]
    mean_control = mean_control.toarray().ravel()[0]

# Pseudocount to avoid div by zero
pseudocount = 1e-9
log2fc = ((mean_perturb-mean_control)) / np.log(2)

logfc = (mean_perturb) - (mean_control)

print(f'Log2FC for {gene_name} in {cell_type}: {log2fc}')
print(f'LogFC for {gene_name} in {cell_type}: {logfc}')


Log2FC for Tafa1 in L6 CT CTX: 5.516055283501741
LogFC for Tafa1 in L6 CT CTX: 3.8234381675720215


In [39]:
import numpy as np

gene_name = 'Tafa1'  # replace with your gene of interest
cell_type = 'L6 CT CTX'  # your cell type of interest

# Get gene index
gene_idx = adata.var_names.get_loc(gene_name)

# Filter cells by condition AND cell type
perturb_cells = adata[(adata.obs['Assign'] == 'Tbr1_0') & (adata.obs['predicted.subclass'] == cell_type)]
control_cells = adata[(adata.obs['Assign'] == 'NT_0') & (adata.obs['predicted.subclass'] == cell_type)]

# Calculate mean expression for the gene in each group
mean_perturb = perturb_cells.X[:, gene_idx].mean()
mean_control = control_cells.X[:, gene_idx].mean()

# Handle sparse matrix if needed
if hasattr(mean_perturb, "toarray"):
    mean_perturb = mean_perturb.toarray().ravel()[0]
    mean_control = mean_control.toarray().ravel()[0]

# Pseudocount to avoid div by zero
pseudocount = 1e-9
log2fc = np.log2((mean_perturb + pseudocount) / (mean_control + pseudocount))

logfc = np.log((mean_perturb + pseudocount) / (mean_control + pseudocount))

print(f'Log2FC for {gene_name} in {cell_type}: {log2fc}')
print(f'LogFC for {gene_name} in {cell_type}: {logfc}')

Log2FC for Tafa1 in L6 CT CTX: 3.497027812574385
LogFC for Tafa1 in L6 CT CTX: 2.423954968625648


In [3]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)


In [4]:
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)

In [5]:
adata.var.head(2)

,variable_gene,highly_variable,means,dispersions,dispersions_norm
Rp1,False,True,0.003160,0.783008,1.411100
Sox17,True,True,0.000658,2.186958,5.102882


In [5]:
# Store the current index (gene names) into a new column called 'gene_name'
adata.var['gene_name'] = adata.var.index

# Optionally reset the index if you want to replace it with default integers
adata.var.reset_index(drop=True, inplace=True)

In [6]:
# Show the unique cell types
adata.obs['Assign'].value_counts()

Assign
Bcl11b_0    5367
Trio_0      4274
Dab1_0      4123
Tbr1_0      3466
NT_0        2735
Rb1cc1_0    2534
Satb2_0     2320
Herc1_0     2165
NT_1        1979
Ank3_0      1763
Xpo7_0      1602
Cx3cl1_0    1392
Cul1_0      1297
ST_0         307
ST_1         203
Name: count, dtype: int64

In [ ]:
def clean_assign(val):
    if val in {'NT_0', 'NT_1', 'ST_0', 'ST_1'}:
        return 'ctrl'
    else:
        return val.replace('_0', '').capitalize()

adata.obs['Assign_clean'] = adata.obs['Assign'].apply(clean_assign)

In [7]:
def clean_assign(val):
    if val in {'NT_0'}:
        return 'ctrl'
    else:
        return val.replace('_0', '').capitalize()

adata.obs['Assign_clean'] = adata.obs['Assign'].apply(clean_assign)

to_remove = ['Nt_1', 'St', 'St_1']
adata = adata[~adata.obs['Assign_clean'].isin(to_remove)].copy()

In [8]:
# Show the unique cell types
adata.obs['Assign_clean'].value_counts()

Assign_clean
Bcl11b    5367
Trio      4274
Dab1      4123
Tbr1      3466
ctrl      2735
Rb1cc1    2534
Satb2     2320
Herc1     2165
Ank3      1763
Xpo7      1602
Cx3cl1    1392
Cul1      1297
Name: count, dtype: int64

In [9]:
adata.obs['condition'] = adata.obs['Assign_clean'].apply(
    lambda x: x if x == 'ctrl' else f"{x.split('+')[0].upper()}+ctrl"
)

In [10]:
adata.obs['condition'].value_counts()

condition
BCL11B+ctrl    5367
TRIO+ctrl      4274
DAB1+ctrl      4123
TBR1+ctrl      3466
ctrl           2735
RB1CC1+ctrl    2534
SATB2+ctrl     2320
HERC1+ctrl     2165
ANK3+ctrl      1763
XPO7+ctrl      1602
CX3CL1+ctrl    1392
CUL1+ctrl      1297
Name: count, dtype: int64

In [ ]:
adata.obs.head(2)

In [11]:
print(adata.obs['predicted.subclass'].value_counts())

predicted.subclass
L6 CT CTX        7860
L4/5 IT CTX      3761
L5 PT CTX        3710
L2/3 IT CTX-1    3122
L5 NP CTX        2755
L6 IT CTX        2544
L5 IT CTX        1939
CT SUB           1689
Pvalb            1317
L2/3 IT CTX-2    1120
Sst               838
NP SUB            468
L3 RSP-ACA        370
L6b CTX           367
L2/3 IT PPP       356
L5 IT TPE-ENT     172
L5 PPP            155
Vip               116
Lamp5             111
Oligo              52
Sncg               51
SUB-ProS           33
Car3               30
Astro              29
NP PPP             26
CR                 14
Micro-PVM           9
L2 IT RHP           7
Endo                6
SMC-Peri            5
VLMC                2
L2 IT ENTl          1
L6 IT ENTl          1
L6b/CT ENT          1
V3d                 1
Name: count, dtype: int64


In [13]:
print(adata.obs['predicted.subclass'].nunique())

35


In [12]:
to_keep = ['L6 CT CTX']
adata_l6 = adata[adata.obs['predicted.subclass'].isin(to_keep)].copy()

In [15]:
adata_l6

AnnData object with n_obs × n_vars = 7860 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition'
    var: 'variable_gene', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'gene_name'
    uns: 'log1p', 'hvg'

In [13]:
adata_l6.obs['condition'].value_counts()

condition
BCL11B+ctrl    1320
DAB1+ctrl      1019
TRIO+ctrl      1000
ctrl            689
HERC1+ctrl      579
RB1CC1+ctrl     561
TBR1+ctrl       559
SATB2+ctrl      551
ANK3+ctrl       469
CUL1+ctrl       385
XPO7+ctrl       369
CX3CL1+ctrl     359
Name: count, dtype: int64

In [14]:
adata_l6.obs['cell_type'] = 'L6 CT CTX'

In [15]:
adata_l6.obs['cell_type'].value_counts()

cell_type
L6 CT CTX    7860
Name: count, dtype: int64

In [22]:
print(os.getcwd())


/gpfs/group/jin/asun/GEARS/0_boli


In [20]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'boli_l6_ct_norm_specific', adata = adata_l6) # specific dataset name and adata object
pert_data.load(data_path = './data/boli_l6_ct_norm_specific') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1, test_perts="TBR1+ctrl_SATB2+ctrl",only_test_set_perts=True) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
  0%|          | 0/12 [00:00<?, ?it/s]

TRIO+ctrl


 58%|█████▊    | 7/12 [00:17<00:09,  1.93s/it]

DAB1+ctrl


 83%|████████▎ | 10/12 [00:24<00:04,  2.20s/it]

CX3CL1+ctrl


100%|██████████| 12/12 [00:28<00:00,  2.36s/it]
Done!
Saving new dataset pyg object at ./data/boli_l6_ct_norm_specific/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/boli_l6_ct_norm_specific/splits/boli_l6_ct_norm_specific_simulation_single_1_0.75_TBR1+ctrl_SATB2+ctrl.pkl
Done!


KeyError: 'test'

In [22]:
test_perts = "TBR1+ctrl_SATB2+ctrl"
if test_perts:
    test_perts = test_perts.split('_')

In [23]:
test_perts

['TBR1+ctrl', 'SATB2+ctrl']

In [27]:
pert_data.adata.obs["split"].value_counts()

split
train    4931
val       551
Name: count, dtype: int64

In [1]:
import importlib
import gears.data_utils 
import gears

importlib.reload(gears.data_utils)
importlib.reload(gears)

from gears import PertData


/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: Futur

In [32]:
from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'boli_l6_ct_norm_specific', adata = adata_l6) # specific dataset name and adata object
pert_data.load(data_path = './data/boli_l6_ct_norm_specific') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1, test_perts="TBR1+ctrl_SATB2+ctrl",only_test_set_perts=True) # get data split with seed

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
  0%|          | 0/12 [00:00<?, ?it/s]

TRIO+ctrl


 58%|█████▊    | 7/12 [00:16<00:09,  1.88s/it]

DAB1+ctrl


 83%|████████▎ | 10/12 [00:24<00:04,  2.21s/it]

CX3CL1+ctrl


100%|██████████| 12/12 [00:28<00:00,  2.35s/it]
Done!
Saving new dataset pyg object at ./data/boli_l6_ct_norm_specific/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/boli_l6_ct_norm_specific/splits/boli_l6_ct_norm_specific_simulation_single_1_0.75_TBR1+ctrl_SATB2+ctrl.pkl
Done!


In [33]:
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

KeyError: 'test'

In [7]:
import importlib
import gears.data_utils
import gears.pertdata
import gears

# Step 1: Reload dependencies first (bottom-up)
importlib.reload(gears.data_utils)
importlib.reload(gears.pertdata)
importlib.reload(gears)  # This re-runs __init__.py

# Step 2: Re-import PertData (must be done *after* reload)
from gears import PertData

In [6]:
%%bash
rm -rf /gpfs/home/asun/jin_lab/GEARS/0_boli/data/boli_l6_ct_norm_specific/splits/*.pkl

In [8]:
pert_data = PertData('./data') # specific saved folder
pert_data.load(data_path = './data/boli_l6_ct_norm_specific') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1, test_perts="TBR1+ctrl_SATB2+ctrl",only_test_set_perts=True) # get data split with seed

Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/boli_l6_ct_norm_specific/splits/boli_l6_ct_norm_specific_simulation_single_1_0.75_TBR1+ctrl_SATB2+ctrl.pkl
Done!


['ANK3' 'BCL11B' 'CUL1' 'HERC1' 'RB1CC1' 'SATB2' 'TBR1' 'XPO7']
['SATB2' 'TBR1']
['ANK3' 'BCL11B' 'CUL1' 'HERC1' 'RB1CC1' 'XPO7']
['ANK3' 'BCL11B' 'CUL1' 'HERC1' 'RB1CC1' 'XPO7']


In [9]:
pert_data.set2conditions

{'test': ['TBR1+ctrl', 'SATB2+ctrl'],
 'train': ['BCL11B+ctrl',
  'ctrl',
  'ANK3+ctrl',
  'CUL1+ctrl',
  'RB1CC1+ctrl',
  'HERC1+ctrl'],
 'val': ['XPO7+ctrl']}